# Demonstrating SQL Injection: A Hands-on Experience


## Purpose of this notebook

- This notebook provides a practical demonstration of the SQL injection vulnerability.
- You'll see a vulnerable web application built with Flask and SQLite in action, learn how an attacker can exploit it, and then see how to fix the vulnerability using secure coding practices.


## Target Audience

- This demo is designed for both technical and non-technical professionals to gain a tangible understanding of SQL injection risks and prevention methods.


## Items covered in the notebook

1. Set up a simple SQLite database.
2. Run a vulnerable Flask application.
3. Manually exploit the vulnerability using a common SQL injection technique.
4. Run a secure version of the application that prevents SQL injection.
5. Attempt the same attack on the secure application and observe the prevention.


## Display Mermaid Diagrams


In [1]:
# This cell contains code to allow mermaid diagrams to be displayed in Jupyter notebooks.
# Run this code cell so that the mermaid diagrams in the notebook are displayed correctly.
# https://mermaid.js.org/ecosystem/tutorials.html#jupyter-integration-with-mermaid-js
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

## Step 1: Creating the Database

- This step sets up our SQLite database with a `users` table and a sample user.
- Running the cell below will create a file named `users.db` in the same directory.


In [ ]:
import sqlite3

DATABASE = 'users.db'

def create_database():
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL,
            password TEXT NOT NULL
        )
    ''')

    cursor.execute("INSERT OR IGNORE INTO users (username, password) VALUES (?, ?)", ('testuser', 'password123'))

    conn.commit()
    conn.close()
    print(f"Database '{DATABASE}' created and populated.")

if __name__ == '__main__':
    create_database()

## Step 2: The Vulnerable Application

- This Flask application contains a login form that is **vulnerable to SQL injection**. We'll examine why shortly.


In [ ]:
from flask import Flask, request, render_template_string
import sqlite3
import socket

app = Flask(__name__)

DATABASE = 'users.db'

def get_db_connection():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    return conn

@app.route('/', methods=['GET'])
def login_form():
    return render_template_string("""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Simple Login (Vulnerable)</title>
        </head>
        <body>
            <div style="display: flex; justify-content: center; align-items: center; height: 100vh; font-family: sans-serif;">
                <div style="padding: 20px; border: 1px solid #ccc; border-radius: 5px; width: 300px;">
                    <h1 style="text-align: center;">Login</h1>
                    <form action="/authenticate" method="post">
                        <div style="margin-bottom: 10px;">
                            <label for="username" style="display: block; margin-bottom: 5px;">Username:</label>
                            <input type="text" id="username" name="username" style="width: 100%; padding: 8px; box-sizing: border-box; border: 1px solid #ddd; border-radius: 3px;">
                        </div>
                        <div style="margin-bottom: 15px;">
                            <label for="password" style="display: block; margin-bottom: 5px;">Password:</label>
                            <input type="password" id="password" name="password" style="width: 100%; padding: 8px; box-sizing: border-box; border: 1px solid #ddd; border-radius: 3px;">
                        </div>
                        <input type="submit" value="Login" style="width: 100%; padding: 10px; background-color: #007bff; color: white; border: none; border-radius: 3px; cursor: pointer;">
                    </form>
                </div>
            </div>
        </body>
        </html>
    """)

@app.route('/authenticate', methods=['POST'])
def authenticate():
    username = request.form['username']
    password = request.form['password']

    conn = get_db_connection()
    cursor = conn.cursor()

    # VULNERABLE SQL query - string formatting
    # The following line directly embeds user-provided input into the SQL query string.
    # This makes the application vulnerable to SQL injection because an attacker can manipulate
    # the 'username' and 'password' variables to inject malicious SQL code.
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"

    print(f"Executing query: {query}") # Print the executed query for demonstration

    cursor.execute(query)
    results = cursor.fetchall()
    conn.close()

    if len(results) > 0:
        return render_template_string("""
            <!DOCTYPE html>
            <html>
            <head>
                <title>Login Result</title>
            </head>
            <body>
                <div style="display: flex; justify-content: center; align-items: center; height: 100vh; font-family: sans-serif;">
                    <div style="padding: 20px; border: 1px solid #4CAF50; border-radius: 5px; width: 300px; text-align: center;">
                        <h2 style="color: #4CAF50;">Login Successful!</h2>
                    </div>
                </div>
            </body>
            </html>
        """)
    else:
        return render_template_string("""
            <!DOCTYPE html>
            <html>
            <head>
                <title>Login Result</title>
            </head>
            <body>
                <div style="display: flex; justify-content: center; align-items: center; height: 100vh; font-family: sans-serif;">
                    <div style="padding: 20px; border: 1px solid #f44336; border-radius: 5px; width: 300px; text-align: center;">
                        <h2 style="color: #f44336;">Login Failed.</h2>
                    </div>
                </div>
            </body>
            </html>
        """)

if __name__ == '__main__':
    # Create the database and table if they don't exist (for demo purposes)
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL,
            password TEXT NOT NULL
        )
    ''')
    # Add a sample user
    cursor.execute("INSERT OR IGNORE INTO users (username, password) VALUES (?, ?)", ('testuser', 'password123'))
    conn.commit()
    conn.close()

    # Find a free port
    sock = socket.socket()
    sock.bind(('', 0))  # Bind to port 0 to get a free port
    port = sock.getsockname()[1]
    sock.close()

    print(f"Running on port: {port}")
    app.run(debug=False, port=port)

## Step 3: Demonstrating the Vulnerability

Let's see how an attacker can exploit this vulnerable application.

**Instructions:**

1. **Run the code cell above** to start the vulnerable Flask server. Note the port number it runs on (e.g., "Running on port: 5000").
2. **Open a web browser** and go to `http://localhost:<your_port_number>`. You should see the login form.
3. **Normal Login:** Enter "testuser" as the username and "password123" as the password. Click "Login". You should see "Login Successful!"
4. **Attempting the SQL Injection:** Now, in the **username** field, enter the following payload. Leave the password field empty or enter an incorrect password:
   ```text
   ' OR '1'='1' --
   ```
5. Click "Login".

**Observe the Output:** You should see "Login Successful!" even though you didn't enter the correct password.

**Explanation of the Vulnerability:**
The vulnerability lies in this line of the `authenticate` function:

```python
query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
```

- **String Formatting:** This code uses an f-string to directly insert the `username` and `password` values from the form into the SQL query.
- **Lack of Sanitization:** The application doesn't check or clean the user input before putting it into the query. This allows attackers to inject their own SQL code.

**How the Attack Works:**
When you enter `' OR '1'='1' --` as the username, the resulting SQL query becomes:

```sql
SELECT * FROM users WHERE username = '' OR '1'='1' --' AND password = ''
```

- `' OR '1'='1'` : This part makes the `WHERE` clause always true. `'1'='1'` is always true, and the `OR` means the condition is met if either the username matches an empty string (which it effectively does due to the initial quote) or `'1'='1'` is true.
- `--'` : This is an SQL comment. It tells the database to ignore the rest of the line, effectively removing the `AND password = ''` condition.

Because the `WHERE` clause is now always true, the query returns all rows from the `users` table. The application checks if any rows were returned and incorrectly declares the login successful.

This demonstrates how directly embedding unsanitized user input into SQL queries creates a **SQL injection vulnerability**.


In [2]:
mm("""
sequenceDiagram
participant Attacker
participant Browser
participant Flask App (Vulnerable)
participant SQLite Database
Attacker->>Browser: Enters malicious input (' OR '1'='1' --) in username field
Browser->>Flask App (Vulnerable): Sends request with malicious input
Flask App (Vulnerable)->>SQLite Database: Executes unsanitized SQL query
SQLite Database-->>Flask App (Vulnerable): Returns all user records
Flask App (Vulnerable)-->>Browser: Returns 'Login Successful'
Browser-->>Attacker: Displays 'Login Successful'
""")


## Important: Interrupt Cell Execution

- It is crucial to stop the vulnerable Flask server before proceeding.
- Go to the "Kernel" menu at the top of the notebook and select "Interrupt".
- Or, if you ran the cell outside the notebook, find and stop the running Python process.


## Step 4: The Secure Application

This Flask application demonstrates how to prevent SQL injection using **parameterized queries**.


In [ ]:
from flask import Flask, request, render_template_string
import sqlite3
import socket

app = Flask(__name__)

DATABASE = 'users.db'

def get_db_connection():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    return conn

@app.route('/', methods=['GET'])
def login_form():
    return render_template_string("""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Simple Login (Secure)</title>
        </head>
        <body>
            <div style="display: flex; justify-content: center; align-items: center; height: 100vh; font-family: sans-serif;">
                <div style="padding: 20px; border: 1px solid #ccc; border-radius: 5px; width: 300px;">
                    <h1 style="text-align: center;">Login</h1>
                    <form action="/authenticate" method="post">
                        <div style="margin-bottom: 10px;">
                            <label for="username" style="display: block; margin-bottom: 5px;">Username:</label>
                            <input type="text" id="username" name="username" style="width: 100%; padding: 8px; box-sizing: border-box; border: 1px solid #ddd; border-radius: 3px;">
                        </div>
                        <div style="margin-bottom: 15px;">
                            <label for="password" style="display: block; margin-bottom: 5px;">Password:</label>
                            <input type="password" id="password" name="password" style="width: 100%; padding: 8px; box-sizing: border-box; border: 1px solid #ddd; border-radius: 3px;">
                        </div>
                        <input type="submit" value="Login" style="width: 100%; padding: 10px; background-color: #007bff; color: white; border: none; border-radius: 3px; cursor: pointer;">
                    </form>
                </div>
            </div>
        </body>
        </html>
    """)

@app.route('/authenticate', methods=['POST'])
def authenticate():
    username = request.form['username']
    password = request.form['password']

    conn = get_db_connection()
    cursor = conn.cursor()

    # Secure SQL query using parameterized query.
    # The '?' are placeholders for the username and password.
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    # The user-provided values are passed separately as a tuple,
    # ensuring they are treated as data, not SQL code.
    cursor.execute(query, (username, password))

    print(f"Executing query: {query} with parameters: {(username, password)}") # Show the query and parameters

    results = cursor.fetchall()
    conn.close()

    if len(results) > 0:
        return render_template_string("""
            <!DOCTYPE html>
            <html>
            <head>
                <title>Login Result</title>
            </head>
            <body>
                <div style="display: flex; justify-content: center; align-items: center; height: 100vh; font-family: sans-serif;">
                    <div style="padding: 20px; border: 1px solid #4CAF50; border-radius: 5px; width: 300px; text-align: center;">
                        <h2 style="color: #4CAF50;">Login Successful!</h2>
                    </div>
                </div>
            </body>
            </html>
        """)
    else:
        return render_template_string("""
            <!DOCTYPE html>
            <html>
            <head>
                <title>Login Result</title>
            </head>
            <body>
                <div style="display: flex; justify-content: center; align-items: center; height: 100vh; font-family: sans-serif;">
                    <div style="padding: 20px; border: 1px solid #f44336; border-radius: 5px; width: 300px; text-align: center;">
                        <h2 style="color: #f44336;">Login Failed.</h2>
                    </div>
                </div>
            </body>
            </html>
        """)

if __name__ == '__main__':
    # Find a free port
    sock = socket.socket()
    sock.bind(('', 0))  # Bind to port 0 to get a free port
    port = sock.getsockname()[1]
    sock.close()

    print(f"Running on port: {port}")
    app.run(debug=False, port=port)

## Step 5: Demonstrating the Prevention

Now, let's see how the secure application prevents SQL injection.

**Instructions:**

1. **Run the code cell above** to start the secure Flask server. Note the port number.
2. **Open a web browser** and go to `http://localhost:<your_port_number>`. You'll see the same login form, but this time it's secure.
3. **Normal Login:** Enter "testuser" and "password123". Click "Login". You should see "Login Successful!"
4. **Attempting the SQL Injection:** Enter the same malicious payload in the **username** field again:
   ```text
   ' OR '1'='1' --
   ```
5. Leave the password empty or incorrect and click "Login".

**Observe the Output:** You should now see "Login Failed."

**Explanation of the Prevention:**
The secure code uses **parameterized queries**:

```python
query = "SELECT * FROM users WHERE username = ? AND password = ?"
cursor.execute(query, (username, password))
```

- **Placeholders:** Instead of directly inserting the `username` and `password`, the query uses `?` as placeholders.
- **Separate Parameters:** The actual values are passed to `cursor.execute()` as a separate tuple `(username, password)`.

**How Parameterized Queries Prevent SQL Injection:**
When using parameterized queries, the database driver treats the SQL query structure and the input parameters separately.

- The driver knows that the `?` placeholders represent data values.
- It then safely substitutes the provided parameters into the query **without** interpreting them as SQL code.

In the case of the attempted SQL injection, the payload `' OR '1'='1' --` is treated as a literal string for the `username` parameter. The database searches for a username that exactly matches this string, which doesn't exist. Therefore, the query returns no results, and the login fails correctly.

Parameterized queries ensure that user input is always treated as data, effectively preventing attackers from injecting malicious SQL code.


In [3]:
mm("""
sequenceDiagram
    participant Attacker
    participant Browser
    participant Flask App (Secure)
    participant SQLite Database
    Attacker->>Browser: Enters malicious input (' OR '1'='1' --) in username field
    Browser->>Flask App (Secure): Sends request with malicious input
    Flask App (Secure)->>SQLite Database: Executes parameterized SQL query with input as data
    SQLite Database-->>Flask App (Secure): Returns no user records matching the literal username
    Flask App (Secure)-->>Browser: Returns 'Login Failed'
    Browser-->>Attacker: Displays 'Login Failed'
""")

## Important: Interrupt Cell Execution

- Make sure to interrupt the execution of the secure Flask server as well before moving on.

## Conclusion

- This demonstration highlights the critical security difference between vulnerable and secure coding practices when interacting with databases.
- The **vulnerable application** directly embedded user input into SQL queries using string formatting, leaving it wide open to SQL injection attacks.
- The **secure application** effectively prevents SQL injection by using **parameterized queries**. This technique ensures that user-provided input is treated as data and not as executable SQL code.


## Key Takeaway

- Always use parameterized queries (or prepared statements with similar functionality in other database libraries) when working with databases to protect against SQL injection vulnerabilities.
- This simple yet powerful technique is a fundamental aspect of secure web development.